In [40]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
from sklearn import metrics
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import time
from datetime import datetime
'''import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

"import os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [41]:
def extract_date(df,column): #process the date,extract the year,month,day,hour and weekday
    df[column+'_year'] = df[column].apply(lambda x:x.year)
    df[column+'_month'] = df[column].apply(lambda y:y.month)
    df[column+'_day'] = df[column].apply(lambda z:z.day)
    df[column+'_hour'] = df[column].apply(lambda h:h.hour)
    df[column+'_weekday'] = df[column].dt.weekday#0-6

In [42]:
def Weekday(data): #check weekday
        data['if_weekday'] = np.nan 
        data.loc[(data.date_weekday >= 5),'if_weekday'] = 1  #Saturday and Sunday:1
        data.loc[(data.date_weekday <= 4),'if_weekday'] = 0  #other days:0

In [43]:
train = pd.read_csv('/kaggle/input/msbd5001/train.csv',parse_dates = ['date'],dayfirst = True)
test = pd.read_csv('/kaggle/input/msbd5001/test.csv',parse_dates = ['date'],dayfirst = True)
extract_date(train,'date')
extract_date(test,'date')
Weekday(train)
Weekday(test)
df_train = train.drop(columns= ['id','date'], axis= 1).copy()
df_test = test.drop(columns= ['id','date'], axis= 1).copy()
#model
Train, Valid = train_test_split(df_train, test_size = 0.3)     
X_train = Train.drop(['speed'], axis=1)     
Y_train = Train["speed"]                    
X_valid = Valid.drop(['speed'], axis=1)       
Y_valid = Valid["speed"]                      
Y_valid = Y_valid.reset_index().drop('index',axis = 1)
Y_train = Y_train.reset_index().drop('index',axis = 1)
dtrain = xgb.DMatrix(X_train, label=Y_train)    
dvalid = xgb.DMatrix(X_valid, label=Y_valid)      
dtest = xgb.DMatrix(df_test)                
xgb_pars = {
        'nthread':            4,  
        'objective':          'reg:squarederror',
        'learning_rate':      0.01,
        'min_child_weight':   0.7,  
        'max_depth':          10,  
        'subsample':          0.87,
        'colsample_bytree':   1,
        'colsample_bylevel':  0.68,
        'verbosity':             0,
        'reg_alpha':          0.2,
        'gamma':              0.1,
        'eval_metric':        'rmse',
    }

watchlist = [(dtrain, 'train'), (dvalid, 'valid')]
model = xgb.train(xgb_pars, dtrain, 3000, watchlist, early_stopping_rounds=5,maximize=False, verbose_eval=1)
print('Modeling MSE %.5f' % model.best_score**2)
#fscores = model.get_fscore()
pred = model.predict(dtest)
result = pd.DataFrame({"id":range(3504), "speed":pred})
result.to_csv("submission_new.csv", index=False)

[0]	train-rmse:34.75623	valid-rmse:34.47997
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 5 rounds.
[1]	train-rmse:34.41478	valid-rmse:34.14081
[2]	train-rmse:34.07708	valid-rmse:33.80542
[3]	train-rmse:33.74438	valid-rmse:33.47495
[4]	train-rmse:33.41337	valid-rmse:33.14625
[5]	train-rmse:33.08659	valid-rmse:32.82144
[6]	train-rmse:32.76280	valid-rmse:32.49965
[7]	train-rmse:32.44154	valid-rmse:32.18029
[8]	train-rmse:32.12847	valid-rmse:31.86932
[9]	train-rmse:31.81408	valid-rmse:31.55701
[10]	train-rmse:31.50283	valid-rmse:31.24759
[11]	train-rmse:31.19502	valid-rmse:30.94165
[12]	train-rmse:30.89001	valid-rmse:30.63854
[13]	train-rmse:30.58826	valid-rmse:30.33881
[14]	train-rmse:30.29025	valid-rmse:30.04296
[15]	train-rmse:29.99422	valid-rmse:29.74907
[16]	train-rmse:29.70146	valid-rmse:29.45800
[17]	train-rmse:29.41148	valid-rmse:29.16984
[18]	train-rmse:29.12443	valid-rmse:28.88458
[19]	train-

[180]	train-rmse:6.87857	valid-rmse:6.87385
[181]	train-rmse:6.82948	valid-rmse:6.82654
[182]	train-rmse:6.78114	valid-rmse:6.78035
[183]	train-rmse:6.73292	valid-rmse:6.73367
[184]	train-rmse:6.68516	valid-rmse:6.68819
[185]	train-rmse:6.63772	valid-rmse:6.64294
[186]	train-rmse:6.59127	valid-rmse:6.59851
[187]	train-rmse:6.54521	valid-rmse:6.55486
[188]	train-rmse:6.49931	valid-rmse:6.51098
[189]	train-rmse:6.45429	valid-rmse:6.46842
[190]	train-rmse:6.41036	valid-rmse:6.42663
[191]	train-rmse:6.36700	valid-rmse:6.38479
[192]	train-rmse:6.32387	valid-rmse:6.34371
[193]	train-rmse:6.28131	valid-rmse:6.30333
[194]	train-rmse:6.23939	valid-rmse:6.26334
[195]	train-rmse:6.19764	valid-rmse:6.22383
[196]	train-rmse:6.15717	valid-rmse:6.18475
[197]	train-rmse:6.11575	valid-rmse:6.14645
[198]	train-rmse:6.07495	valid-rmse:6.10824
[199]	train-rmse:6.03571	valid-rmse:6.07107
[200]	train-rmse:5.99635	valid-rmse:6.03390
[201]	train-rmse:5.95779	valid-rmse:5.99726
[202]	train-rmse:5.91903	valid-r

[367]	train-rmse:3.24816	valid-rmse:3.70513
[368]	train-rmse:3.24269	valid-rmse:3.70148
[369]	train-rmse:3.23664	valid-rmse:3.69798
[370]	train-rmse:3.23186	valid-rmse:3.69551
[371]	train-rmse:3.22590	valid-rmse:3.69189
[372]	train-rmse:3.21889	valid-rmse:3.68736
[373]	train-rmse:3.21269	valid-rmse:3.68356
[374]	train-rmse:3.20679	valid-rmse:3.67974
[375]	train-rmse:3.20047	valid-rmse:3.67508
[376]	train-rmse:3.19402	valid-rmse:3.67095
[377]	train-rmse:3.18918	valid-rmse:3.66828
[378]	train-rmse:3.18337	valid-rmse:3.66438
[379]	train-rmse:3.17806	valid-rmse:3.66151
[380]	train-rmse:3.17360	valid-rmse:3.65938
[381]	train-rmse:3.16884	valid-rmse:3.65673
[382]	train-rmse:3.16182	valid-rmse:3.65275
[383]	train-rmse:3.15621	valid-rmse:3.64967
[384]	train-rmse:3.15077	valid-rmse:3.64716
[385]	train-rmse:3.14441	valid-rmse:3.64390
[386]	train-rmse:3.13966	valid-rmse:3.64104
[387]	train-rmse:3.13457	valid-rmse:3.63794
[388]	train-rmse:3.12985	valid-rmse:3.63465
[389]	train-rmse:3.12546	valid-r

[554]	train-rmse:2.58529	valid-rmse:3.42484
[555]	train-rmse:2.58237	valid-rmse:3.42393
[556]	train-rmse:2.58073	valid-rmse:3.42409
[557]	train-rmse:2.57876	valid-rmse:3.42351
[558]	train-rmse:2.57709	valid-rmse:3.42299
[559]	train-rmse:2.57419	valid-rmse:3.42262
[560]	train-rmse:2.57192	valid-rmse:3.42194
[561]	train-rmse:2.56820	valid-rmse:3.42130
[562]	train-rmse:2.56527	valid-rmse:3.42021
[563]	train-rmse:2.56212	valid-rmse:3.41907
[564]	train-rmse:2.55804	valid-rmse:3.41738
[565]	train-rmse:2.55493	valid-rmse:3.41618
[566]	train-rmse:2.55237	valid-rmse:3.41562
[567]	train-rmse:2.54992	valid-rmse:3.41439
[568]	train-rmse:2.54750	valid-rmse:3.41408
[569]	train-rmse:2.54587	valid-rmse:3.41367
[570]	train-rmse:2.54192	valid-rmse:3.41213
[571]	train-rmse:2.53928	valid-rmse:3.41193
[572]	train-rmse:2.53647	valid-rmse:3.41133
[573]	train-rmse:2.53402	valid-rmse:3.41090
[574]	train-rmse:2.53020	valid-rmse:3.41025
[575]	train-rmse:2.52768	valid-rmse:3.40962
[576]	train-rmse:2.52588	valid-r

[741]	train-rmse:2.12261	valid-rmse:3.33342
[742]	train-rmse:2.12013	valid-rmse:3.33292
[743]	train-rmse:2.11831	valid-rmse:3.33297
[744]	train-rmse:2.11646	valid-rmse:3.33253
[745]	train-rmse:2.11431	valid-rmse:3.33231
[746]	train-rmse:2.11235	valid-rmse:3.33213
[747]	train-rmse:2.10956	valid-rmse:3.33139
[748]	train-rmse:2.10768	valid-rmse:3.33085
[749]	train-rmse:2.10595	valid-rmse:3.33061
[750]	train-rmse:2.10344	valid-rmse:3.33017
[751]	train-rmse:2.10116	valid-rmse:3.33017
[752]	train-rmse:2.09933	valid-rmse:3.32996
[753]	train-rmse:2.09764	valid-rmse:3.32987
[754]	train-rmse:2.09532	valid-rmse:3.32939
[755]	train-rmse:2.09386	valid-rmse:3.32898
[756]	train-rmse:2.09188	valid-rmse:3.32858
[757]	train-rmse:2.08994	valid-rmse:3.32816
[758]	train-rmse:2.08857	valid-rmse:3.32779
[759]	train-rmse:2.08638	valid-rmse:3.32793
[760]	train-rmse:2.08452	valid-rmse:3.32742
[761]	train-rmse:2.08325	valid-rmse:3.32734
[762]	train-rmse:2.08103	valid-rmse:3.32710
[763]	train-rmse:2.07867	valid-r